In [3]:
import numpy as np
import pandas as pd
from math import sqrt
from copy import deepcopy
import matplotlib.pyplot as plt

In [5]:
from sklearn.datasets import load_wine

from sklearn.manifold import TSNE

In [6]:
data = load_wine()
data.target[[10, 80, 140]]

array([0, 1, 2])

In [7]:
list(data.target_names)

['class_0', 'class_1', 'class_2']

In [8]:

tsne = TSNE(n_components=2, random_state=0)
X = tsne.fit_transform(data.data)

In [10]:
#finding Kmax
kmax=int(sqrt(X.shape[0]))
kmax

13

In [12]:
#global centered
cen=np.array(np.mean(X))
print(cen)

0.10509296


In [13]:
# Euclidean Distance Caculator
def dist(a, b, ax=1):
    return np.linalg.norm(a - b, axis=ax)


In [14]:
# calculate AvgDistance
def AvgDist(a):
    sum=0;
    for i in range(len(a)):
        for j in range(len(a)):
            if(j > i):
                #print(i,j)
                sum=sum+dist(a[i],a[j],None)
        #print(" ")
    #print("total sum is:",sum)
    AvgDist=(2/(len(a)*(len(a)-1)))*sum
    
    return AvgDist

In [15]:
#calculating epsilon neighbour value
epsilon=0.125*AvgDist(X)
print(epsilon)

1.5391514807928373


In [16]:
# calculating density parameter
def DensityParameter(a):
    S=[]
    
    for i in range(len(a)):
        total=0
        for j in range(len(a)):
            if(i != j):
                sum=epsilon - dist(a[i],a[j],None)
                if (sum >=0):
                    sum=1
                else:
                    sum=0
                total=total+sum
        S.append(total)
    #index_max = np.argmax(S)
    #print (S)
    return S#a[index_max]       
   

In [17]:
#Storing density paremeter value 
dpl=DensityParameter(X)
#print(final)

In [18]:
VCVI=np.zeros(kmax)

In [19]:
for k in range(2,kmax):
    final=dpl.copy()
    #creating an array for storing initial cenroid
    a=[]
    #finding point with highest density parameter value
    for i in range(k):
        index_max=np.argmax(final)
#        print("index number:", index_max)
#        print("original value at that index:",X[index_max])
        a.append(X[index_max])
        final[index_max]=-1
        for j in range(len(X)):
            if (dist(a[i],X[j],None) < epsilon):
                final[j]=-1
                
    #Initial centroids
    C=np.array(list(a),dtype=np.float32)
    print("Initial Centroids for k=",k)
    print(C)
    # To store the value of centroids when it updates
    C_old = np.zeros(C.shape)
#    print(C_old)
# Cluster Lables(0, 1, 2)
    clusters = np.zeros(len(X))
#    print(clusters)

# Error func. - Distance between new centroids and old centroids
    error=dist(C,C_old,None)
#    print(error)
# Loop will run till the error becomes zero

    while error != 0:
        # Assigning each value to its closest cluster
        for i in range(len(X)):
            distances = dist(X[i], C)
           # print("distance=",distances,"X[i]=",X[i],"C=",C)
            cluster = np.argmin(distances)
           # print("cluster=",cluster)
            clusters[i] = cluster
        # Storing the old centroid values
        C_old = deepcopy(C)
       # print("old centroid=",C_old)
    
        # Finding the new centroids by taking the average value
        for i in range(k):
            points = [X[j] for j in range(len(X)) if clusters[j] == i]
            C[i] = np.mean(points, axis=0)
        error = dist(C, C_old, None)
#    print("new centroid=",C)
#    count=count+1
#    print("count=",count)
    difference=0
    for i in range(k):
        difference=difference+dist(cen,C[i],None)
#    print("cen=",cen,"C[i]=",C[i])
#    print(difference)
    G=(difference/k)
    #calculating Ti
    total=0
    for i in range(k):
            points = np.array([X[j] for j in range(len(X)) if clusters[j] == i])
            for j in range(len(points)):
#                print("point=",points[j],"center=",C[i],"distance=",dist(points[j],C[i],None))
                total=total+dist(points[j],C[i],None)
#        print(" ")
    T=total/X.shape[0]
    VCVI[k]=((k*k*G)+((X.shape[0])*T))/X.shape[0]

Initial Centroids for k= 2
[[ 8.421882  -6.6177344]
 [12.633137  -9.498085 ]]
Initial Centroids for k= 3
[[ 8.421882  -6.6177344]
 [12.633137  -9.498085 ]
 [-8.535162   6.5206394]]
Initial Centroids for k= 4
[[  8.421882   -6.6177344]
 [ 12.633137   -9.498085 ]
 [ -8.535162    6.5206394]
 [-11.714783    9.086669 ]]
Initial Centroids for k= 5
[[  8.421882   -6.6177344]
 [ 12.633137   -9.498085 ]
 [ -8.535162    6.5206394]
 [-11.714783    9.086669 ]
 [-10.267678    8.382047 ]]
Initial Centroids for k= 6
[[  8.421882    -6.6177344 ]
 [ 12.633137    -9.498085  ]
 [ -8.535162     6.5206394 ]
 [-11.714783     9.086669  ]
 [-10.267678     8.382047  ]
 [  0.21373458  -0.16870546]]
Initial Centroids for k= 7
[[  8.421882    -6.6177344 ]
 [ 12.633137    -9.498085  ]
 [ -8.535162     6.5206394 ]
 [-11.714783     9.086669  ]
 [-10.267678     8.382047  ]
 [  0.21373458  -0.16870546]
 [ 11.030436    -8.450217  ]]
Initial Centroids for k= 8
[[  8.421882    -6.6177344 ]
 [ 12.633137    -9.498085  ]
 [

In [20]:
min=VCVI[2]
min

4.819833936651102

In [21]:
for k in range(3,kmax):
    if(min > VCVI[k]):
        min=VCVI[k]
        kopt=k
    

In [22]:
print(kopt)
print(VCVI[kopt])

4
2.993459685464923


In [24]:
print(VCVI[4])

2.993459685464923
